<a href="https://colab.research.google.com/github/Papagoat/algo-trading-strategies/blob/main/Momentum_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credit to [Algovibes](https://www.youtube.com/watch?v=5W_Lpz1ZuTI)

In [ ]:
pip install yfinance

In [ ]:
from dateutil.relativedelta import relativedelta
import datetime as dt
import numpy as np
import pandas as pd
import pandas_datareader as reader
import requests
import yfinance as yf

# Get list of crypto tickers from Yahoo Finance

In [ ]:
crypto_list = 'http://sg.finance.yahoo.com/cryptocurrencies'

In [ ]:
tickersCrypto = pd.read_html(crypto_list)[0].Symbol.to_list()

# Download candlestick chart using yfinance

In [ ]:
start = dt.datetime(2018,1,31)
end = dt.datetime.now()

df = yf.download(tickersCrypto, start=start, end=end)
df = df['Adj Close']

[*********************100%***********************]  25 of 25 completed


# Calculate monthly returns by cumulating daily returns

In [ ]:
mtl_ret = df.pct_change().resample('M').agg(lambda x:(x+1).prod() -1)
mtl_ret

# Calculate returns over the past 11 months

In [ ]:
past_11 = (mtl_ret+1).rolling(11).apply(np.prod)-1
past_11.dropna()

In [ ]:
formation = dt.datetime(2021,5,31)
end_measurement = formation - relativedelta(months=1)

In [ ]:
ret_12 = past_11.loc[end_measurement]
ret_12 = ret_12.reset_index()

In [ ]:
ret_12['quintile'] = pd.qcut(ret_12.iloc[:,1],5,labels=False)
ret_12.columns = ['Symbols', 'Date', 'quintile']

In [ ]:
winners = ret_12[ret_12.quintile >= 3]
losers = ret_12[ret_12.quintile == 0]

In [ ]:
winner_ret = mtl_ret.loc[formation + relativedelta(months=1), df.columns.isin(winners.Symbols)]
loser_ret = mtl_ret.loc[formation + relativedelta(months=1), df.columns.isin(losers.Symbols)]

In [ ]:
momentum_profit = winner_ret.mean() - loser_ret.mean()

In [ ]:
Crypto = yf.download(tickersCrypto, start=start, end=end)['Adj Close'].pct_change().resample('M').agg(lambda x: (x+1).prod() - 1)